In [1]:
import torch
import sys
from models.anomaly_transformer import get_anomaly_transformer 

original_gelu_repr = torch.nn.GELU.__repr__
torch.nn.GELU.__repr__ = lambda self: "GELU()"

model_path = "logs/best_checkpoints/SWaT_parameters.pt"
model_old = torch.load(model_path, map_location='cpu')

print(model_old)

c:\Users\victo\anaconda3\envs\BERTTS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AnomalyTransformer(
  (linear_embedding): Linear(in_features=700, out_features=512, bias=True)
  (transformer_encoder): TransformerEncoder(
    (encoder_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention_layer): MultiHeadAttentionLayer(
          (word_fc_layers): ModuleList(
            (0-2): 3 x Linear(in_features=512, out_features=512, bias=True)
          )
          (output_fc_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (feed_forward_layer): PositionWiseFeedForwardLayer(
          (first_fc_layer): Linear(in_features=512, out_features=2048, bias=True)
          (second_fc_layer): Linear(in_features=2048, out_features=512, bias=True)
          (activation_layer): GELU()
          (dropout_layer): Dropout(p=0.1, inplace=False)
        )
        (norm_layers): ModuleList(
          (0-1): 2 x LayerNorm((512,), eps=1e-06, elementwise_affine=True)
        )
        (dropout_layer): Dropout(p=0.1, inplace=False)
      )
    )
  )
 

c:\Users\victo\anaconda3\envs\BERTTS\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
C:\Users\victo\AppData\Local\Temp\ipykernel_18768\315758326.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializatio

In [2]:
def convert_model_to_state_dict(model_path, state_dict_path):
    try:

        loaded_object = torch.load(model_path, map_location=torch.device('cpu'))
        print(str(type(loaded_object)))

        print(f"Extraction du state_dict depuis le modèle complet dans '{model_path}'.")
        state_dict = loaded_object.state_dict()

        torch.save(state_dict, state_dict_path)
        print(f"Le state_dict a été sauvegardé avec succès dans '{state_dict_path}'.")

    except Exception as e:
        print(f"Une erreur est survenue : {e}", file=sys.stderr)

def get_model_anomaly(parameter_path, input_dim, output_dim, patch_size, max_seq_len, transformer_layers, device):

    model = get_anomaly_transformer(
        input_d_data=input_dim,
        output_d_data=output_dim, 
        patch_size=patch_size,
        max_seq_len=max_seq_len,
        transformer_n_layer=transformer_layers
    )
    model.to(device)
    print("Architecture créée.")

    try:
        state_dict = torch.load(parameter_path, map_location=device, weights_only=True)
        model.load_state_dict(state_dict)
        model.eval()
        
        print("\n Modèle chargé")

        return model

    except Exception as e:
        print(f"\n Erreur Chargement : {e}")

In [3]:
output_state_dict_path = 'state_dict_SWAT.pt'
parameter_path = "./state_dict_SWAT.pt"

input_dim = 50        
patch_size = 14        
max_seq_len = 512      
transformer_layers = 6 
output_dim = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Utilisation du device : {device}")

convert_model_to_state_dict(model_path, output_state_dict_path)
model_new = get_model_anomaly(parameter_path=parameter_path,
                            input_dim=input_dim,
                            output_dim=output_dim,
                            patch_size=patch_size,
                            max_seq_len=max_seq_len,
                            transformer_layers=transformer_layers,
                            device=device)

Utilisation du device : cuda
<class 'models.anomaly_transformer.AnomalyTransformer'>
Extraction du state_dict depuis le modèle complet dans 'logs/best_checkpoints/SWaT_parameters.pt'.
Le state_dict a été sauvegardé avec succès dans 'state_dict_SWAT.pt'.


C:\Users\victo\AppData\Local\Temp\ipykernel_18768\627359956.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_object = torch.load(model_path, map_location=torch.dev

Architecture créée.

 Modèle chargé


In [4]:
torch.save(model_new,"model_SWAT.pt")